### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [11]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F751E27250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F751E275B0>, model_name='Gemma2-9b-It')

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

message = [
    SystemMessage(content="Translate the sentence, from English to Telugu"),
    HumanMessage(content="My hometown is Kakinada. I am a Software engineer and work in Hyderabad")
]

In [13]:
model.invoke(message)

AIMessage(content='Here\'s the translation of the sentence, along with some notes on Telugu grammar:\n\n**Telugu:**\n\nనా పుట్టిన పట్టణం క\u200c akinada. నేను సాఫ్ట్ వేర్ ఇంజినీర్ గా పని చేస్తున్నాను హైదరాబాద్ లో. \n\n**Pronunciation:**\n\n* nā puṭṭina paṭṭaṇaṁ Kakinada. \n* nēnu sāfṭa wēra injinēra gā paṇi cēṣṭunnaanu haiḍarābāḍ lo.\n\n**Explanation:**\n\n* **నా (nā):**  "My"\n* **పుట్టిన పట్టణం (puṭṭina paṭṭaṇaṁ):** "Hometown" \n* **క\u200c akinada (Kakinada):** Your hometown name \n* **నేను (nēnu):** "I"\n* **సాఫ్ట్ వేర్ ఇంజినీర్ (sāfṭa wēra injinēra):** "Software Engineer"\n* **గా (gā):** "As a"\n* **పని చేస్తున్నాను (paṇi cēṣṭunnaanu):** "am working"\n* **హైదరాబాద్ (haiḍarābāḍ):** "Hyderabad"\n* **లో (lo):** "In"\n\n\nLet me know if you have any other sentences you\'d like translated!\n', response_metadata={'token_usage': {'completion_tokens': 336, 'prompt_tokens': 32, 'total_tokens': 368, 'completion_time': 0.610909091, 'prompt_time': 0.001486368, 'queue_time': 0.250542182, 'tota

In [14]:
result = model.invoke(message)
result.content

"Here's the translation, broken down for clarity:\n\n* **My hometown is Kakinada.**  \n    * నా స్వగ్రామం కాకినాడ. (Naa swagramem Kakinada.)\n\n* **I am a Software engineer and work in Hyderabad.**\n    * నేను సాఫ్ట్\u200cవేర్ ఇంజనీర్ గా పనిచేస్తున్నాను హైదరాబాద్ లో. (Neenu saaftswaer engineer ga panichestuunanu Hyderabad lo.)\n\n**Combined:**\nనా స్వగ్రామం కాకినాడ. నేను సాఫ్ట్\u200cవేర్ ఇంజనీర్ గా పనిచేస్తున్నాను హైదరాబాద్ లో. \n\n\n\nLet me know if you have any other sentences you'd like translated! \n"

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

"Here's the translation, broken down for clarity:\n\n* **My hometown is Kakinada.**  \n    * నా స్వగ్రామం కాకినాడ. (Naa swagramem Kakinada.)\n\n* **I am a Software engineer and work in Hyderabad.**\n    * నేను సాఫ్ట్\u200cవేర్ ఇంజనీర్ గా పనిచేస్తున్నాను హైదరాబాద్ లో. (Neenu saaftswaer engineer ga panichestuunanu Hyderabad lo.)\n\n**Combined:**\nనా స్వగ్రామం కాకినాడ. నేను సాఫ్ట్\u200cవేర్ ఇంజనీర్ గా పనిచేస్తున్నాను హైదరాబాద్ లో. \n\n\n\nLet me know if you have any other sentences you'd like translated! \n"

In [16]:
### Using LCEL- chain the components

chain = model | parser
chain.invoke(message)

'Here\'s the translation, along with some notes on natural phrasing:\n\n**Telugu:**\n\n* నా స్వగత ప్రాంతం క\u200c akkinada. నేను సాఫ్ట్\u200cవేర్ ఇంజనీర్ గా ప\u200cని చేస్తున్నాను, హైదరాబాద్ లో.\n\n**Pronunciation:**\n\n*  naa swagat praantham kakinada. neenu saafwaree ingeener ga pani chesthunnaanu, haiderabad lo.\n\n**Explanation:**\n\n* **నా స్వగత ప్రాంతం (naa swagat praantham):** My hometown\n* **క\u200c akkinada (kakinada):** Kakinada\n* **నేను (neenu):** I \n* **సాఫ్ట్\u200cవేర్ ఇంజనీర్ (saafwaree ingeener):** Software engineer\n* **గా (ga):** as\n* **ప\u200cని చేస్తున్నాను (pani chesthunnaanu):** I am working\n* **హైదరాబాద్ (haiderabad):** Hyderabad\n* **లో (lo):** in\n\n**Notes:**\n\n* The word order in Telugu can be slightly different from English. \n*  "Swagat praantham" is a common way to say "hometown" in Telugu. \n\n\n\n\nLet me know if you have any other sentences you\'d like translated!\n'

In [17]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "You are now a translator. So please translate the below text to {language}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{input_text}")
    ]
)

input = "Hey Hi! How are you?"
result_1 = prompt.invoke({"language":"Telugu","input_text":input})
result_1.to_messages()

[SystemMessage(content='You are now a translator. So please translate the below text to Telugu'),
 HumanMessage(content='Hey Hi! How are you?')]

In [18]:
chain_2 = prompt | model | parser

chain_2.invoke({"language":"Telugu","input_text":input})

"హే! హలో! మీరు ఎలా ఉన్నారు?  (Hey! Hello! How are you?) \n\n\nLet me know if you have any other text you'd like me to translate! \n"